In [ ]:
%load_ext micropython_magic
%reload_ext micropython_magic

In [ ]:
%mpy -s {"/dev/cu.usbmodem2101"}

In [ ]:
# %%micropython

import time
from config import config
from machine import Pin, RTC
from mp_libs.network import Network
from mp_libs.protocols import espnow_protocol
from mp_libs.protocols import min_iot_protocol
from mp_libs.protocols.wifi_protocols import WifiProtocol
from mp_libs.time import ptp

TIMEOUT_MSEC = 5000
TIMESTAMP_BUFFER_SIZE = 8 * 10


# Create network, connect, and ntp sync
# timestamps_buffer = bytearray(TIMESTAMP_BUFFER_SIZE)
# timestamps_idx = 0
ts_buffer = []
event_signal = False
rtc = RTC()
wifi = Network.create_wifi()
miniot_network = Network.create_min_iot("GTY")
epn = miniot_network.transport.transport._transport
wifi.connect()
miniot_network.connect()
wifi.ntp_time_sync()

def pin_callback(pin):
    now = rtc.now()

    # global timestamps_idx, event_signal
    # timestamps_buffer[timestamps_idx:timestamps_idx + 8] = now.to_bytes(8)
    # timestamps_idx = (timestamps_idx + 1) % TIMESTAMP_BUFFER_SIZE
    global event_signal
    ts_buffer.append(now)
    event_signal = True

pin = Pin(37, mode=Pin.IN)
pin.irq(handler=pin_callback, trigger=Pin.IRQ_RISING)

print(f"Initial RTC datetime: {rtc.datetime()}")
print(f"Initial RTC now: {rtc.now()}")
print(f"My MAC: {epn.wifi._sta.config('mac')}")
print(f"EPN Peer: {config['epn_peer_mac']}")
print(f"Channel: {epn.wifi._sta.config('channel')}")
print(f"EPN timeout: {epn.epn.config('timeout_ms')}")

# Handle scan
rxed_packets = []
data_available = False
while not data_available:
    data_available = epn.receive(rxed_packets)
data_available = False

while True:
    rxed_packets = []
    data_available = False

    # Wait for signal from controller
    while event_signal is False:
        time.sleep_ms(1)
    event_signal = False

    # Extract timestamp and print it
    # timestamp = int.from_bytes(timestamps_buffer[timestamps_idx:timestamps_idx + 8])
    timestamp = ts_buffer.pop(-1)
    print(f"Device A Event Timestamp: {timestamp}")

    # Perform PTP cycle
    # Receive data
    while not data_available:
        data_available = miniot_network.receive(rxed_packets)
        time.sleep_ms(1)

    # Parse rx'ed packets
    for packet in rxed_packets:
        if ptp.is_ptp_msg(packet.msg):
            ptp_type, payload = ptp.parse_msg(packet.msg)
        else:
            print(f"Rx'ed unexpected message: {packet}")
            continue

        # Perform PTP sync
        if ptp_type == ptp.PtpMsg.SYNC_REQ:
            ptp.sequence_master(
                miniot_network,
                lambda miniot_msg: miniot_msg.msg,
                TIMEOUT_MSEC,
                num_sync_cycles=payload)

            # print(f"now: {rtc.datetime()}")
            # print(f"RTC now: {rtc.now()}")

    while True:
        # Wait for signal from controller
        while event_signal is False:
            time.sleep_ms(1)
        event_signal = False

        # Extract timestamp and print it
        # timestamp = int.from_bytes(timestamps_buffer[timestamps_idx:timestamps_idx + 8])
        timestamp = ts_buffer.pop(-1)
        print(f"Device A Event Timestamp: {timestamp}")
